# ICL data science challenge

## EDA

In [148]:
import pandas as pd
import numpy as np
import re

In [ ]:
train_path = "../input/raw/train.csv"
preprocessed_path = "../input/derived/train-preprocessed.csv"
cols = None
data_lines = []

with open(train_path, "rt") as fin:
    with open(preprocessed_path, "wt") as fout:
        for line in fin:
            fout.write(line.replace('\\"', "'"))

In [149]:
df = pd.read_csv(preprocessed_path, sep=r'\s+', quotechar='"')
df

,Index,Artist,Track,Album,danceability,energy,loudness,speechiness,acousticness,instrumentalness,...,AnC,AnD,BnC,BnD,CnD,AnBnC,AnBnD,AnCnD,BnCnD,AnBnCnD
0,1,tajima hal,Grape Choice,Grape Choice,0.704,0.5870,-9.815,0.3120,0.341,0.004510,...,0,0,0,0,0,0,0,0,0,0
1,2,CHROMANCE,Wrap Me In Plastic - Marcus Layton Radio Edit,Wrap Me In Plastic (Marcus Layton Radio Edit),0.917,0.6120,-2.183,0.0854,0.238,0.000023,...,0,0,0,0,0,0,0,0,0,0
2,3,Stevie Wonder,Sir Duke,Songs In The Key Of Life,0.588,0.5680,-9.766,0.0672,0.160,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,4,MishCatt,Midnight Sun - Acoustic Version,Midnight Sun (Acoustic Version),0.590,0.0991,-15.925,0.0490,0.915,0.000005,...,0,0,0,0,0,0,0,0,0,0
4,5,Gloria Estefan,Sólo Por Tu Amor,Alma Caribeña,0.561,0.4770,-8.123,0.0355,0.295,0.000000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7588,7589,Some Sprouts,Someone You Love,Divinely Uninspired To A Hellish Extent,0.501,0.4050,-5.679,0.0319,0.751,0.000000,...,0,0,0,0,0,0,0,0,0,0
7589,7590,Sure Sure,Funky Galileo,Funky Galileo,0.802,0.3250,-9.327,0.0466,0.164,0.162000,...,0,0,0,0,0,0,0,0,0,0
7590,7591,The Flying Stars Of Brooklyn NY,Live On,My God Has a Telephone,0.513,0.6260,-8.843,0.0582,0.579,0.000633,...,0,0,0,0,0,0,0,0,0,0
7591,7592,Gasper Lawal,Kita Kita,Nigeria 70 - Funky Lagos,0.614,0.9070,-6.565,0.0472,0.062,0.772000,...,0,0,0,0,0,0,0,0,0,0


## Sanity checking cells

In [150]:
temp = ['AnB', 'AnC', 'AnD', 'BnC', 'BnD', 'CnD',
        'AnBnC', 'AnBnD', 'AnCnD', 'BnCnD', 'AnBnCnD']
for i in range(len(df['A'])):
    for c in temp:
        row = df.iloc[i]
        if row[c] == 1:
            if 'A' in c:
                if row['A'] == 0:
                    print(row['Index'], 'A')
            if 'B' in c:
                if row['B'] == 0:
                    print(row['Index'], 'B')
            if 'C' in c:
                if row['C'] == 0:
                    print(row['Index'], 'C')
            if 'D' in c:
                if row['D'] == 0:
                    print(row['Index'], 'D')

234 A
330 B
752 A
2016 B
2261 A
2419 A
3531 B
3863 B
3873 B
3873 B
4375 A
4997 B
5230 B
5466 B
6337 A
6362 A
6627 B
7372 B
7432 A


In [151]:
df.columns

Index(['Index', 'Artist', 'Track', 'Album', 'danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'tempo', 'length', 'time_signature', 'A', 'B', 'C', 'D',
       'AnB', 'AnC', 'AnD', 'BnC', 'BnD', 'CnD', 'AnBnC', 'AnBnD', 'AnCnD',
       'BnCnD', 'AnBnCnD'],
      dtype='object')

## Baseline model logistic regression

In [152]:
df.columns
feature_cols = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'tempo', 'length']
target_cols = ['A', 'B', 'C', 'D']

X = df[feature_cols].to_numpy()
y = df[target_cols].to_numpy()

In [153]:
X.shape, y.shape

((7593, 9), (7593, 4))

In [154]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

logRegs = [XGBClassifier(objective='binary:logistic', random_state=42), LogisticRegression(), LogisticRegression(), LogisticRegression()]
accs = []

for i in range(4):
    logRegs[i].fit(X_train, y_train[:, i])
    preds = logRegs[i].predict(X_test)
    acc = accuracy_score(y_test[:, i], preds)
    accs.append(acc)
    print(f"Validation accuracy for class {target_cols[i]}: {acc:.4f}")
print(f"Average validation accuracy: {np.mean(accs)}")

Validation accuracy for class A: 0.6656
Validation accuracy for class B: 0.6274
Validation accuracy for class C: 0.9835
Validation accuracy for class D: 0.8308
Average validation accuracy: 0.7768268597761685


In [155]:
df_test = pd.read_csv('testanswer.csv')

In [156]:
X_test = df_test[feature_cols].to_numpy()
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)
y_test = df_test[target_cols].to_numpy()

In [158]:
for i in range(4):
    preds = logRegs[i].predict(X_test)
    acc = accuracy_score(y_test[:, i], preds)
    accs.append(acc)
    print(f"Validation accuracy for class {target_cols[i]}: {acc:.4f}")

Validation accuracy for class A: 0.6270
Validation accuracy for class B: 0.6375
Validation accuracy for class C: 0.9810
Validation accuracy for class D: 0.8135


In [159]:
def fill_remaining_cols(df):
    extended_cols = ['AnB', 'AnC', 'AnD', 'BnC', 'BnD', 'CnD', 'AnBnC', 'AnBnD', 'AnCnD', 'BnCnD', 'AnBnCnD']
    for i in range(len(df['A'])):
        for c in extended_cols:
            fill_val = True
            for l in ['A', 'B', 'C', 'D']:
                if l in c:
                    fill_val = fill_val and (df.at[i, l] == 1)
            # modify the cell
            df.at[i, c] = int(fill_val)
    return(df)

def submit_dataframe(df, submit_path):
    df = df.reset_index()
    df = fill_remaining_cols(df)
    assert len(df.columns) == 29
    df.to_csv(submit_path, index=False, quotechar='"', sep=' ')
            
"../output/2-standard-deviations-submission-1.csv"

'../output/2-standard-deviations-submission-1.csv'

In [160]:
submit_dataframe(df_test, 'sigma.csv')

In [118]:
holi = fill_remaining_cols(df_test.reset_index())
sum(holi['AnB'])

84

In [98]:
df_test = df_test.reset_index()

In [124]:
df_test['A']

1      1
2      0
3      1
4      1
5      0
      ..
945    1
946    0
947    1
948    0
949    1
Name: A, Length: 949, dtype: int32

In [93]:
fill_remaining_cols(df_test)

,index,Artist,Track,Album,danceability,energy,loudness,speechiness,acousticness,instrumentalness,...,AnC,AnD,BnC,BnD,CnD,AnBnC,AnBnD,AnCnD,BnCnD,AnBnCnD
0,1,Son Mieux,Can’t Get Enough,The Mustard Seed,0.625,0.794,-6.168,0.0475,0.415000,0.000282,...,0,0,0,0,0,0,0,0,0,0
1,2,Taste,Blister On The Moon,Taste,0.393,0.588,-15.232,0.0400,0.014400,0.010200,...,0,0,0,0,0,0,0,0,0,0
2,3,Allen Toussaint,Last Train - Remastered Version,Southern Nights,0.581,0.781,-8.890,0.0443,0.119000,0.018900,...,0,0,0,0,0,0,0,0,0,0
3,4,STR4TA,We Like It,Aspects,0.586,0.804,-9.570,0.0398,0.029300,0.644000,...,0,0,0,0,0,0,0,0,0,0
4,5,Scary Pockets,Doo Wop (That Thing) / Just A Friend / Ex-Fact...,Doo Wop (That Thing) / Just A Friend / Ex-Fact...,0.748,0.425,-11.208,0.1210,0.601000,0.000081,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,945,The 1975,Lostmyhead,"I like it when you sleep, for you are so beaut...",0.341,0.671,-7.481,0.0391,0.241000,0.863000,...,0,0,0,0,0,0,0,0,0,0
945,946,Klô Pelgag,Insomnie,Brol La Suite,0.496,0.339,-10.512,0.0426,0.650000,0.000049,...,0,0,0,0,0,0,0,0,0,0
946,947,Barenaked Ladies,Smile - Acoustic,Grinning Streak (Deluxe Version),0.637,0.505,-7.332,0.0406,0.268000,0.000007,...,0,0,0,0,0,0,0,0,0,0
947,948,Skillet,Awake and Alive,Awake,0.434,0.939,-2.983,0.0593,0.042500,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [63]:
len(df_test.columns)

28